In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from diffusers import DiffusionPipeline
import torch
import matplotlib.pyplot as plt
from qunet import QUNet2DConditionModel
from diffusers.models import UNet2DConditionModel
from QPyTorch.qtorch import BlockFloatingPoint, FixedPoint, FloatingPoint
from qpipe import run_qpipe, parse_quant, get_qnet

from torchmetrics.functional.multimodal import clip_score
from functools import partial
from PIL import Image
import os
import numpy as np
from utils.presentation import plot_hybrid_grid, suppress_plot_show, create_video, print_matrices
from utils.evaluate import eval_mse_stats, eval_mse_matrix

In [ ]:
directory = "images"
##directory = "images_20_v4"

def search_directory(directory):
    runs = os.listdir(directory)
    runs = [os.path.join(directory, run) for run in runs if os.path.isdir(os.path.join(directory, run))]
    return runs

runs = search_directory(directory)




In [ ]:
mruns = []
conds=["M23E8", "M3E4"]
for cond in conds:
    mruns = mruns + [run for run in runs if cond in run]


filters = ["morgana2","x800"]
for filter in filters:
    mruns =  [run for run in mruns if filter in run]

def swapPositions(list, pos1, pos2):
     
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list

mruns


In [ ]:
runs = ['images/morgana2x100_M23E8_V4',
        'images/morgana2x100_A_M23E8_W_M3E4_flex_V4',
        'images/morgana2x100_M3E4_flex_V4',
        'images/morgana2x100_M3E4_flex_V4_embedding_',
        'images/morgana2x100_M3E4_flex_V4_embedding__adjustedV2',
]
torch.cuda.set_device(1)
create_video("100_steps_M3E4_embedding_adjustments", runs, 4, display_mse = True, display_is = True, repeat_baseline = True)

In [ ]:
runs = [   
 'images/morgana2x800_M23E8_V4',
 'images/morgana2x800_M2E5_V4',
 'images/morgana2x800_M2E5_flex_V4',
 ##'images/morgana2x800_M2E5_flex_embedding',
 'images/morgana2x800_M2E5_flex_not_embedding',
 'images/morgana2x800_M2E5_flex_embedding_adjustedV2',
 'images/sim_morgana2x800_M2E5_flex_embedding_adjustedV2',
]
create_video("800_steps_M2E5_embedding_adjustments", runs, 4, seconds=5, display_mse = True, display_is = False, repeat_baseline = True)

In [ ]:
runs = ['images/morgana2x800_M23E8_V4',
 'images/morgana2x800_M3E4_V4',
 'images/morgana2x800_M3E4_flex_V4',
 'images/morgana2x800_M3E4_flex_V4_not_embedding_',
 'images/morgana2x800_M3E4_flex_V4_embedding_',
 'images/morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
]
torch.cuda.set_device(1)
create_video("800_steps_M3E4_embedding_adjustments", runs, 4, display_mse = True, display_is = False, repeat_baseline = True)

In [ ]:
from utils.evaluate import inception_score, from_dirs

images_png = from_dirs(['images/morgana2x800_M3E4_V4'], [0], type = "png")
inception_score(images_png, batch_size=1, splits = 1)[0] 

In [ ]:
runs = [
    'images/morgana2x800_M23E8_V4',
    'images/morgana2x800_M3E4_flex_V4_embedding_',
    'images/morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
    'images/sim_morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
    ]

create_video("800_steps_M3E4_embedding_adjusted", runs, 4)

In [ ]:
runs = [
    'images/morgana2x800_M23E8_V4',
    'images/morgana2x800_M3E4_flex_V4_embedding_',
    'images/morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
    'images/sim_morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
    ]

create_video("800_steps_M3E4_embedding_adjusted", runs, 4)

In [ ]:
runs = [
    'images/morgana2x800_M23E8_V4',
    'images/morgana2x800_M3E4_flex_V4_embedding_',
    'images/morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
    'images/sim_morgana2x800_M3E4_flex_V4_embedding__adjustedV2',
    ]

##with suppress_plot_show():
plot_hybrid_grid("800_steps_M3E4_embedding_adjusted", runs, 4, show_mse = True)

In [ ]:
runs = [
    'images/sunsetx800_M23E8_V4',
    'images/sunsetx800_M3E4_flex_V4',
    'images/sunsetx800_M3E4_flex_V4_adjustedV2',
    'images/sim_sunsetx800_M3E4_flex_V4_adjustedV2',
    ]

with suppress_plot_show():
    plot_hybrid_grid("800_steps_M3E4", runs, 4)

In [ ]:
qtype = "M3E4"
            
mruns = [
 f'images/M23E8_V4',
 f'images/A_M23E8_W_{qtype}_flex_V4',
 f'images/{qtype}_flex_V4',
 f'images/{qtype}_flex_V4_adjustedV2',
 f'images/sim_{qtype}_flex_V4_adjustedV2',
#  f'images/22_{qtype}_flex_V4',
#  f'images/22_{qtype}_flex_V4_adjustedV2',
#  f'images/sim_22_{qtype}_flex_V4_adjustedV2',
 f'images/25_{qtype}_flex_V4',
 f'images/25_{qtype}_flex_V4_adjustedV2',
 f'images/sim_25_{qtype}_flex_V4_adjustedV2']


with suppress_plot_show():
    plot_hybrid_grid(qtype, mruns, [0,1,2,3]);

In [ ]:

qtype = "M3E4"
            
mruns = [
 f'images/M23E8_V4',
 f'images/A_M23E8_W_{qtype}_flex_V4',
 f'images/{qtype}_flex_V4',
 f'images/{qtype}_flex_V4_adjustedV2',
 f'images/sim_{qtype}_flex_V4_adjustedV2',
#  f'images/22_{qtype}_flex_V4',
#  f'images/22_{qtype}_flex_V4_adjustedV2',
#  f'images/sim_22_{qtype}_flex_V4_adjustedV2',
 f'images/25_{qtype}_flex_V4',
 f'images/25_{qtype}_flex_V4_adjustedV2',
 f'images/sim_25_{qtype}_flex_V4_adjustedV2']


with suppress_plot_show():
    plot_hybrid_grid(qtype, mruns, [0,1,2,3], directory="images_20_v4")

In [ ]:
qtype = "M2E5"


mruns = [
 'images_20_v4/M23E8_V3',
 'images_20_v4/M2E5_flex_V3',   
 'images_20_v4/24_M2E5_flex_V3',
 'images_20_v4/24_M2E5_flex_V4_adjustedV2',
 'images_20_v4/25_M2E5_flex_V3',
 'images_20_v4/25_M2E5_flex_V4_adjustedV2',
 'images_20_v4/25_M23E8_V3']

with suppress_plot_show():
    plot_hybrid_grid(qtype, mruns, [0,1,2,3])

In [ ]:
qtype = "M2E5"


mruns = [
    'images/M23E8_V4',
    'images/A_M23E8_W_M2E5_flex_V4',
    'images/20_M2E5_flex_V4',
    'images/20_M2E5_flex_V4_adjustedV2',
    'images/25_M2E5_flex_V4',
    'images/25_M2E5_flex_V4_adjustedV2',
    'images/30_M2E5_flex_V4',
    'images/30_M2E5_flex_V4_adjustedV2',
    ]

with suppress_plot_show():
    plot_hybrid_grid(qtype, mruns, [0,1,2,3])


In [ ]:
runs = search_directory("images")


mruns = []
conds=["morgana"]
for cond in conds:
    mruns = mruns + [run for run in runs if cond in run]

mruns

In [ ]:
runs100 = [
    'images/morgana2x100_M23E8_V4',
    'images/morgana2x100_M3E4_flex_V4',
    'images/morgana2x100_M4E3_flex_V4',
    'images/morgana2x100_M2E5_flex_V4',
    ]

with suppress_plot_show():
    plot_hybrid_grid("100steps", runs100, [0,1,2,3])

In [ ]:
runs = [
    'images/morgana2x20_M23E8_V4',
    'images/morgana2x20_M2E5_flex_V4',
    'images/morgana2x100_M23E8_V4',
    'images/morgana2x100_M2E5_flex_V4',
    'images/morgana2x300_M2E5_V4',
    'images/morgana2x300_M2E5_V4_adjustedV2',
    'images/sim_morgana2x300_M2E5_V4_adjustedV2',
    'images/morgana2x400_M23E8_V4',
    'images/morgana2x400_M2E5_flex_V4',
    'images/morgana2x400_M2E5_flex_V4_adjustedV2',
    'images/sim_morgana2x400_M2E5_flex_V4_adjustedV2',
    ]

with suppress_plot_show():
    plot_hybrid_grid("more_steps_M2E5", runs, [0,1,2,3])